<a href="https://colab.research.google.com/github/Carapet07/Data-Science-projects/blob/main/First_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##My first NLP project

In [1]:
import os, pathlib, shutil, random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  41.6M      0  0:00:01  0:00:01 --:--:-- 41.5M
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drin

In [3]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

# Move 20% of the training data into new validation directory
for category in ("neg", "pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files) # shuffle files for randomness
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]

  for fname in val_files:
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)

In [4]:
batch_size = 32

# Create datasets from directories, automatically batching text samples
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [5]:
for text_batch, label_batch in train_ds.take(1):  # Take one batch
    for i in range(2):  # Show the first 5 examples
        print(f"Text: {text_batch.numpy()[i].decode('utf-8')}")
        print(f"Label: {label_batch.numpy()[i]}")
        print("=" * 50)


Text: As part of our late 1950s vocabulary, we well knew the Ponderosa, Little Joe, Hoss, Ben Cartwright,etc. on that great show "Bonanza."<br /><br />It came Saturday night and everyone was glued to the television set. This was a real show depicting family values. There may have been a weekly crisis, but it was the strong family atmosphere that pulled everyone together.<br /><br />Lorne Greene was dominant as the patriarch of the family. His words depicted wisdom. We often were left to wonder that Ben Cartwright, a widower, must have been the best of husbands to that poor wife of his who had died. He reared wonderful sons.<br /><br />Naturally, we all wondered why Pernell Roberts left the show. The show was a gold mine and Roberts surrendered loads of money when he departed. His career never took off as he was associated as a Cartwright son. He should have tried to get back into the series. He certainly lost a bonanza by dropping out.
Label: 1
Text: Okay, here's what I think about Jac

In [6]:
max_length = 600 # max length of a movie review
max_tokens = 20000 # max number of unique words to consider

# Create a TextVectorization layer that converts text into integer sequences
# eg: "dog": [0.2, 0.5, -0.7, 0.21... embedding_dim]
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=max_length
)

# Extract only the text fron train_ds, removing lables (0, 1)
text_only_train_ds = train_ds.map(lambda x, y: x)

# This line adapts the TextVectorization layer to the dataset, learning the vocabulary
# by assigning each word in the dataset a unique integer index
text_vectorization.adapt(text_only_train_ds)

# apply it to every dataset we have
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)

In [7]:
for inputs, targets in train_ds:
  print("input shape: ", inputs.shape)
  print("input dtype: ", inputs.dtype)
  print("targets shape: ", targets.shape)
  print("targets dtype: ", targets.dtype)
  print('inputs[0]: ', inputs[0])
  print('targets[0]: ', targets[0])
  break

input shape:  (32,)
input dtype:  <dtype: 'string'>
targets shape:  (32,)
targets dtype:  <dtype: 'int32'>
inputs[0]:  tf.Tensor(b"OK, let's get this clear. I'm really not into sci-fi, but for some reason I love Stargate SG-1. <br /><br />Jack O'Neil takes his team SG-1 through a Stargate. A round device that creates a wormhole. It gives you the ability to travel to distant worlds. It might sound like your usual sci-fi-series, but it's not! The plot is set today not in some distant millennium like many other sci-fi-series. I find that great. It gives you things, happenings and such you can relate to, and you can jump into the series at any time without having to learn many new terms and names of all the gadgets. They have some of course but thanks to O'Neil who likes to keep a simple terminology, there's not many. <br /><br />The series has a nice blending of action, humor and drama. If you enjoy loads of special effects you're not going to find it here. They don't use many bad ones bu

In [8]:
# downlaod GloVe precomputed database for words ebedding task
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2025-02-21 19:00:17--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-02-21 19:00:17--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-02-21 19:00:17--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

Parse the unzipped file (.txt file) to build an index that maps words(as strings) to their words reprezentation

In [9]:
glove_100d_path = "glove.6B.100d.txt"
# 1: Create an embedding dictionary
embeddings_index = {}

# 2: Parse the GloVe txt file. Add each word and it's embedding into emeddings_index dictionary
with open(glove_100d_path) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ") # converts coefs strings into float32
    embeddings_index[word] = coefs

print(f'found {len(embeddings_index)} words')

found 400000 words


Next lets build an embedding matrix that will be loaded into an Embedding layer

In [10]:
embedding_dim = 100

# 1: Get the vocabulary from the TextVectorizaton layer
vocabulary = text_vectorization.get_vocabulary()

# 2: Assign to each word a uniquie number
word_index = dict(zip(vocabulary, range(len(vocabulary))))

# 3: Create a matrix filled with zeros
embedding_matrix = np.zeros((max_tokens, embedding_dim))

# 4: Populate embedding_matrxi with GloVe embeddinngs for each word in the vocabulary
for word, i in word_index.items():
  if i < max_tokens:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector


In [11]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True
)

This dataset yields inputs that are tf.string tensors and targets that are int32 tensors encoding the value "0" or "1"


All set, now let's try learnign something from this data


In [12]:
inputs = keras.Input(shape=(None, ), dtype='int64')
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, None, 256)      │      5,120,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None)           │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 64)             │         73,984 │ embedding_1[0][0],     │
│ (Bidirectional)           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 64)             │              0 │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 1)              │             65 │ dropout[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,194,049 (19.81 MB)

 Trainable params: 5,194,049 (19.81 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                      save_best_only=True)
    ]

history = model.fit(int_train_ds.cache(),
  validation_data=int_val_ds.cache(),
  epochs=10,
  callbacks=callbacks)

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 45s 59ms/step - accuracy: 0.6771 - loss: 0.5759 - val_accuracy: 0.8608 - val_loss: 0.3284
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.8666 - loss: 0.3228 - val_accuracy: 0.8726 - val_loss: 0.2969
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 43ms/step - accuracy: 0.9027 - loss: 0.2506 - val_accuracy: 0.8774 - val_loss: 0.2904
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 31s 50ms/step - accuracy: 0.9218 - loss: 0.2057 - val_accuracy: 0.8808 - val_loss: 0.3010
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.9445 - loss: 0.1533 - val_accuracy: 0.8796 - val_loss: 0.3290
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.9585 - loss: 0.1155 - val_accuracy: 0.8754 - val_loss: 0.3475
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 44ms/step - accuracy: 0.9704 - loss: 0.0858 - val_accuracy: 0.8682 - val_loss: 0.4046
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 43ms/step - accuracy: 0.9772 - loss: 0.0663 - 

Now let's try to use a Transforer architecture

All the code I've taken from the book 'Deep learning in Python' by François Chollet

In [14]:
class TransformerEncoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention = layers.MultiHeadAttention(
        key_dim=embed_dim, num_heads=num_heads
    )
    self.dense_proj = keras.Sequential([
        layers.Dense(dense_dim, activation='relu'),
        layers.Dense(embed_dim),
    ])
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()

  def call(self, inputs, mask=None):
    """
    Mask helps the model ignore padding tokens
    MHA layers expects the mask to be in 3 dimensions, however ours have only 2
   (batch_size, sequence_length). Thus we add a 1 extra dimension so the new shape will be
    (batch_size, 1, sequence_length)
    """
    if mask is not None:
      mask = mask[:, tf.newaxis, :]
    attention_output = self.attention(
        inputs, inputs, attention_mask=mask
    )
    proj_input = self.layernorm_1(inputs + attention_output)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input + proj_output)

  def get(self):
    """
    This function saves the models config, making it possible to reload it later
    """
    config = super().get_config()
    config.update({
        'embeded_dim': self.embed_dim,
        'dense_dim': self.dense_dim,
        'num_heads': self.num_heads
    })
    return config


In [15]:
vocab_size = 20000
dense_dim = 32
num_heads = 2
embed_dim = 200

inputs = layers.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, None, 200)           │       4,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder                  │ (None, None, 200)           │         335,232 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 200)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             201 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,335,433 (16.54 MB)

 Trainable params: 4,335,433 (16.54 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
callbacks = [
    keras.callbacks.ModelCheckpoint('transformer_encoder.keras',
                                    save_best_only=True)
]

history=model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model(
 "transformer_encoder.keras",
 custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 75s 104ms/step - accuracy: 0.6367 - loss: 0.7150 - val_accuracy: 0.8182 - val_loss: 0.3933
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 70s 97ms/step - accuracy: 0.8290 - loss: 0.3863 - val_accuracy: 0.8196 - val_loss: 0.4082
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 61s 98ms/step - accuracy: 0.8519 - loss: 0.3419 - val_accuracy: 0.8562 - val_loss: 0.3272
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 83s 99ms/step - accuracy: 0.8651 - loss: 0.3204 - val_accuracy: 0.8608 - val_loss: 0.3222
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 62s 99ms/step - accuracy: 0.8754 - loss: 0.3002 - val_accuracy: 0.8622 - val_loss: 0.3128
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 100ms/step - accuracy: 0.8856 - loss: 0.2761 - val_accuracy: 0.8628 - val_loss: 0.3129
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 84s 103ms/step - accuracy: 0.8958 - loss: 0.2550 - val_accuracy: 0.8650 - val_loss: 0.3105
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 64s 102ms/step - accuracy: 0.9029 - loss: 0.238

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'transformer_encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8734 - loss: 0.2915
Test acc: 0.865


In [17]:
model = keras.models.load_model(
 "transformer_encoder.keras",
 custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.8728 - loss: 0.2927
Test acc: 0.865


That's a pretty good accuracy we have got. However there is one small problem. The vanilla Transformer we implemented treats sequences as sets, mening it ingnores word order.

For example we have 2 sentences: "I love Star Wars" and "Star love I Wars". The model will divide them into similar sets: "I" "I love" "love" "love Star" "Star Wars" "Wars". Thus on the both sets it will get the same accuracy and won't understand that the first sentence is wrong.

That is why I am going to implement an easy but powerful techniquie called Positional Encoding. It adds a position-dependent vector to each words embeddings.

How will it work in practice?
1. First, we get a word embedding for each word in the sequence
2. Then, we compute the positional encoding for each position
3. We add positional encoding to the word embedding before passing it to the Transformer
Final Input = Word Embedding + Positional Encoding

There are 2 main types of positional encoding:
1. Fixed Positional Encoding (Sinusoidal Encoding)
2. Learned Positional Embeddings (Positional Embedding)
I am going to use the second one. Instead of using predifined mahtematical function, it lets the model learn positional encodings just like word embeddings


In [18]:
class PositionalEmbedding(layers.Layer):
  def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
    super().__init__(**kwargs)
    # embedding for storing token indices
    self.token_embedding = layers.Embedding(
        input_dim=input_dim, output_dim=output_dim
    )
    # embedding for storing the token positions
    self.positinal_embedding = layers.Embedding(
        input_dim=sequence_length, output_dim=output_dim
    )
    self.sequence_length = sequence_length
    self.input_dim = input_dim
    self.output_dim = output_dim

  def call(self, inputs):
    # inputs is a tensor of shape (batch_size, sequence_length)
    # so inputs[-1] reuturns the last dimension: sequence_length
    length = tf.shape(inputs)[-1]
    positions = tf.range(start=0, limit=length, delta=1)
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions = self.positional_embedding(positions)
    return embedded_tokens + embedded_positions

  def mask(self, inputs, mask=None):
    return tf.math.not_equal(inputs, 0)

  def get_config(self):
    config = super().get_config()
    config.update({
        "output_dim": self.output_dim,
        "sequence_length": self.sequence_length,
        "input_dim": self.input_dim
    })
    return config

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.5)
outputs = layers.Dense(1, activation='sigmoid')
model = keras.Model(inputs, outputs)

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy'
    metrics=['accuracy']
)
model.summary()

In [22]:
callbacks = [
    keras.callbacks.ModelCheckpoint('full_transformer_encoder.keras',
                                    save_best_only=True)
]

history=model.fit(
    int_train_ds,
    validation_data=int_val_ds,
    epochs=10,
    callbacks=callbacks
)

model=keras.models.load_model(
    'full_transformer_encoder.keras',
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test accuracy {model.evaluate(int_test_ds)[1]:.3f}")


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 53s 73ms/step - accuracy: 0.9229 - loss: 0.2032 - val_accuracy: 0.8692 - val_loss: 0.3210
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 46s 74ms/step - accuracy: 0.9312 - loss: 0.1785 - val_accuracy: 0.8696 - val_loss: 0.3308
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 74ms/step - accuracy: 0.9355 - loss: 0.1658 - val_accuracy: 0.8702 - val_loss: 0.3310
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 46s 73ms/step - accuracy: 0.9446 - loss: 0.1461 - val_accuracy: 0.8716 - val_loss: 0.3529
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 45s 73ms/step - accuracy: 0.9515 - loss: 0.1278 - val_accuracy: 0.8686 - val_loss: 0.3704
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 44s 71ms/step - accuracy: 0.9590 - loss: 0.1108 - val_accuracy: 0.8680 - val_loss: 0.3987
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 44s 70ms/step - accuracy: 0.9636 - loss: 0.0980 - val_accuracy: 0.8646 - val_loss: 0.4176
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 70ms/step - accuracy: 0.9703 - loss: 0.0847 - 

AttributeError: module 'keras._tf_keras.keras' has no attribute 'load_model'

In [23]:
model=keras.models.load_model(
    'full_transformer_encoder.keras',
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test accuracy {model.evaluate(int_test_ds)[1]:.3f}")=

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'transformer_encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8800 - loss: 0.2879
Test accuracy 0.869


An important takeaway!!!

It turns out that when approaching a new text-classification task, you should pay
close attention to the ratio between the number of samples in your training data and
the mean number of words per sample. If that ratio is small—less
than 1,500—then the bag-of-bigrams model will perform better (and as a bonus, it will
be much faster to train and to iterate on too). If that ratio is higher than 1,500, then
you should go with a sequence model. In other words, sequence models work best
when lots of training data is available and when each sample is relatively short.

So if you’re classifying 1,000-word long documents, and you have 100,000 of them (a
ratio of 100), you should go with a bigram model. If you’re classifying tweets that are
40 words long on average, and you have 50,000 of them (a ratio of 1,250), you should
also go with a bigram model. But if you increase your dataset size to 500,000 tweets (a
ratio of 12,500), go with a Transformer encoder. What about the IMDB movie review
classification task? We had 20,000 training samples and an average word count of 233,
so our rule of thumb points toward a bigram model